Loading packages

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import tensorflow as tf
#Train IELM and evaluating training accuracy
no_classes = 2
error = 0.1
loss_function = "binary_cross_entropy" 
activation_function = "tanh"


Define dataloader for loading of data

In [2]:
def loading_data(train_path,test_path):

    # Define the data transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to (224, 224)
        transforms.ToTensor(),  # Convert images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    ])

    

    train_dataset = ImageFolder(train_path, transform=transform)
    traindata_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

    test_dataset = ImageFolder(test_path, transform=transform)
    testdata_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
    return traindata_loader,testdata_loader


Define model

In [ ]:
import torchvision.models as models
def load_model():
    #loading of model
    device=torch.device("cuda")   
    vgg_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
    vgg_model = torch.nn.Sequential(*list(vgg_model.features.children()))
    vgg_model=vgg_model.to(device)
    vgg_model.eval()
    return vgg_model


Prepare feature Extractor

In [ ]:
#training features extraction
def training_feature_extraction(device, traindata_loader,vgg_model):

    x_training_features = []
    y_train_labels = []
    with torch.no_grad():
        for images, labels in traindata_loader:        
            images = images.to(device)
            labels = labels.to(device)
            # Extract features using VGG16
            with torch.no_grad():
                features = vgg_model(images)   
            
            x_training_features.append(features)
            y_train_labels.append(labels)
    x_training_features = torch.cat(x_training_features, dim=0)
    y_train_labels = torch.cat(y_train_labels, dim=0)

    x_training_features.to(device)
    y_train_labels.to(device)
    return x_training_features,y_train_labels

def testing_feature_extratcion(device,testdata_loader,vgg_model):
    x_testing_features = []
    y_testing_labels = []
    with torch.no_grad():
        for images, labels in testdata_loader:
            images = images.to(device)
            labels = labels.to(device)
            # Extract features using VGG16
            with torch.no_grad():
                features = vgg_model(images) 
            x_testing_features.append(vgg_model(images))
            y_testing_labels.append(labels)
    x_testing_features = torch.cat(x_testing_features, dim=0)
    y_testing_labels = torch.cat(y_testing_labels, dim=0)

    x_testing_features.to(device)
    y_testing_labels.to(device)
    return x_testing_features,y_testing_labels


IRVFLwoDL Class

In [ ]:

"""
IELM Constructor class
"""
import torch
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
class I_RVFL():
    """ Constructor to initialize node"""
    def __init__(self, no_input_nodes, max_no_hidden_nodes, no_output_nodes,
        activation_function='sigmoid', loss_function='mean_squared_error'):
        device=torch.device("cuda") 
        
        self.no_input_nodes = no_input_nodes
        self.no_hidden_nodes = 1
        self.no_output_nodes = no_output_nodes

        self.beta = torch.FloatTensor(self.no_hidden_nodes, self.no_output_nodes).uniform_(-1., 1.)
        self.beta=self.beta.to(device)
        
        # initialize weights between Input Layer and hidden layer
        self.alpha=torch.FloatTensor(self.no_input_nodes, self.no_hidden_nodes).uniform_(-1.,1.)
        self.alpha=self.alpha.to(device)
        
        # Initialize Biases
        self.bias =torch.zeros(size=(self.no_hidden_nodes,))
        self.bias=self.bias.to(device)
        # set an activation function
        self.activation_function = activation_function
       
        # set a loss function
        self.loss_function = loss_function
    
    def mean_squared_error(self,Y_True, Y_Pred):
        return 0.5 * torch.mean((Y_True - Y_Pred)**2)

    def mean_absolute_error(self, Y_True, Y_Pred):
        return torch.mean(torch.abs(Y_True - Y_Pred))
    
    def sigmoid(self, x):
        return 1. /(1.+ torch.exp(-x))
    
    def predict(self, X):
        return torch.tensor(self(X), dtype=torch.float32)
    
    def __call__(self, X):
        h = torch.sigmoid(torch.matmul(X,self.alpha) + self.bias)
        return torch.matmul(h,self.beta)
    
    def binary_cross_entropy(self,Y_true, Y_pred):
        lss=[]
        Y_pred = torch.clamp(Y_pred, min=1e-7, max=1.0 - 1e-7)
        loss_calculated=- (Y_true * torch.log(Y_pred) + (1 - Y_true) * torch.log(1 - Y_pred))
        loss=torch.mean(loss_calculated)
        return loss

    def evaluate(self, X, Y_true, metrics):
        Y_pred =self.predict(X)
        #Y_true = Y_true
        
        ret = []
        
        Y_pred=torch.tensor(Y_pred,dtype=torch.float32)


        for m in metrics:
            if m == 'loss':
                loss=[]
                loss = self.binary_cross_entropy(Y_true, Y_pred)
                ret.append(loss)                
            elif m == 'accuracy':
                Y_pred_argmax = torch.argmax(Y_pred, axis=-1)
                Y_true_argmax = torch.argmax(Y_true, axis=-1)
                acc = torch.sum(Y_pred_argmax == Y_true_argmax) / len(Y_true)
                #plotting of confusion matrix

                # Get predicted probabilities for positive class
                Y_true_argmax=Y_true_argmax.cpu()
                Y_pred_argmax=Y_pred_argmax.cpu()
                self.plot_confusion(Y_true_argmax, Y_pred_argmax)
                ret.append(acc)
            else:
                raise ValueError('an unknown evaluation indicator \'%s\'.' % m)
        if len(ret) == 1:
            print(ret)
            ret = ret
        elif len(ret) == 0:
            ret = None
        return ret

    def fit(self, X, Y_true,Lmax,error,activation):
        device=torch.device("cuda")
        with torch.no_grad():
            self.beta=torch.FloatTensor(np.random.uniform(-1.,1.,size=(1, self.no_output_nodes)))
            self.alpha = torch.FloatTensor(self.no_input_nodes, 1).uniform_(-1,1)
            #print(self.beta.shape, self.alpha.shape,X.shape)
            
            self.alpha=self.alpha.to(device)
            self.beta=self.beta.to(device)
            if(activation=='s'):
                H = torch.sigmoid(torch.matmul(X, self.alpha))
            elif(activation=='t'):
                H = torch.tanh(torch.matmul(X, self.alpha))  
            elif(activation=='r'):
                H = torch.relu(torch.matmul(X, self.alpha))
            
            
            # Compute a pseudoinverse of H
            H_pinv = torch.pinverse(H)

            # Update beta
            self.beta = torch.matmul(H_pinv, Y_true.float())
            
            for i in range(2, Lmax):
                beta_random = torch.FloatTensor(np.random.uniform(-1.,1.,size=(1, self.no_output_nodes)))
                alpha_random = torch.FloatTensor(self.no_input_nodes, 1).uniform_(-1,1)
                
                beta_random=beta_random.to(device)
                alpha_random=alpha_random.to(device)
                
                self.alpha=torch.cat((self.alpha,alpha_random),dim=1)                          
                self.beta =torch.cat((self.beta,beta_random),dim=0)

                if(activation=='s'):
                    H = torch.sigmoid(torch.matmul(X, self.alpha))
                elif(activation=='t'):
                    H = torch.tanh(torch.matmul(X, self.alpha))  
                elif(activation=='r'):
                    H = torch.relu(torch.matmul(X, self.alpha))
                
                H_pinv = torch.pinverse(H)
                self.beta = torch.matmul(H_pinv, Y_true)


In [3]:
device=torch.device("cuda") 
train_path="../Parking_Space_classification/Dataset/Train/"
test_path="D:/Parking_Space_classification/Dataset/Test/"
traindata_loader,testdata_loader=loading_data(train_path,test_path)
model=load_model()

In [7]:
def i_elmTraining(features_train,y_train,activation):
  # ===============================
  # Instantiate ELM object
  # ===============================
  input=features_train.size(1)
  ielm_model = I_ELM(
      no_input_nodes=input,
      max_no_hidden_nodes=Lmax,
      no_output_nodes=no_classes,
      loss_function=loss_function,
      activation_function=activation_function,    
  )

  i = time.time() 
  ielm_model.fit(features_train,y_train,Lmax,error,activation)
  final = time.time()
  return ielm_model
def i_elmTesting(ielm_model,features_test,y_test):
  
  i = time.time()
  test_loss, test_acc = ielm_model.evaluate(features_test, y_test, metrics=['loss', 'accuracy'])
  final = time.time()
  
  print('Testing Loss in mean square error: %f ' % test_loss,"\n")
  print('Testing Accuracy: %f' % test_acc)
  time_taken=(final-i)/features_test.shape[0]
  print('Total Time require for Testing one image is ' ,time_taken ,'seconds')
  


In [ ]:
i_elm_model1=i_elmTraining(x_training_features,y_train_labels,activation)
i_elmTesting(i_elm_model1,x_testing_features,y_testing_labels)